In [22]:
import yfinance as yf
import pandas as pd
import math
import numpy_financial as npf
import statistics
import datetime
from dateutil.relativedelta import relativedelta
from pandas.core.frame import DataFrame
from scipy.stats import gmean


## 設定定期投入金額起始結束日期投資組合配比

In [23]:
start='2020-1-1'
end='2022-12-31'
ratio=[0.5,0.5]
stock=['KO','PYPL']
cashflows=['None','Contribute_fixed']
contribution_amount=1000
contribution_ratio_type=['fixed', 'drift']

In [24]:
PMT=contribution_amount
start_dt=datetime.datetime.strptime(start,"%Y-%m-%d")
start_dt=(start_dt-relativedelta(months=+1)).strftime("%Y-%m-%d")
stock_name=[]
stock_list=[]
end_dt=datetime.datetime.strptime(end,"%Y-%m-%d")
end_dt=(end_dt+datetime.timedelta(days=1)).strftime("%Y-%m-%d")
for i in range(len(stock)):
#     a=round(yf.download(stock[i],start=start_dt,end=end_dt),2)
    a=yf.download(stock[i],start=start_dt,end=end_dt)
    stock_list.append(a)

for i in stock_list:
    i.drop(["Open"],axis=1,inplace=True)
    i.drop(["High"],axis=1,inplace=True)
    i.drop(["Low"],axis=1,inplace=True)
    i.drop(["Close"],axis=1,inplace=True)
    i.drop(["Volume"],axis=1,inplace=True)

if len(stock)==1:
    price=stock_list[0].copy()
    price.rename(columns={'Adj Close':stock[0]},inplace=True)
else:
    price=stock_list[0].copy()
    price.rename(columns={'Adj Close':stock[0]},inplace=True)
    for i in range(1,len(stock)):
        price=pd.merge(price,stock_list[i],left_index=True,right_index=True,how='inner')
        price.rename(columns={'Adj Close':stock[i]},inplace=True)
        
stock_df=price.copy()
stock_m_df=stock_df.resample('BM').last()
stock_m_df=stock_m_df.loc[start_dt:end]
# print(stock_list[0])
stock_m_df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,KO,PYPL
Date,,
2019-12-31,50.008430,108.169998
2020-01-31,52.764091,113.889999
2020-02-28,48.327934,107.989998
2020-03-31,40.330265,95.739998
2020-04-30,41.824989,123.000000
2020-05-29,42.545006,155.009995
2020-06-30,41.092129,174.229996
2020-07-31,43.446560,196.070007
2020-08-31,45.552670,204.139999


In [25]:

TotalCost_list=[]
for i in range(1,len(stock_m_df)+1):
    TotalCost=PMT*i
    TotalCost_list.append(TotalCost)
stock_m_df['total_cost']=TotalCost_list

# 各支股票股數計算
unit_name_list=[]
    
for k in range(len(stock_m_df.columns[0:len(stock)])):
    print(k)
    preNAV_list=[0]
    unit_list=[PMT*ratio[k]/stock_m_df[stock_m_df.columns[k]].iloc[0]]
    unit_name_list.append("unit"+str(k+1))
    for i in stock_m_df[stock_m_df.columns[k]].iloc[1:]:
        unit=unit_list[-1]+PMT*ratio[k]/i
        unit=unit
        unit_list.append(unit)
    stock_m_df[unit_name_list[k]]=unit_list

stock_m_df.head()

0
1


,KO,PYPL,total_cost,unit1,unit2
Date,,,,,
2019-12-31,50.008430,108.169998,1000,9.998314,4.622354
2020-01-31,52.764091,113.889999,2000,19.474456,9.012555
2020-02-28,48.327934,107.989998,3000,29.820439,13.642613
2020-03-31,40.330265,95.739998,4000,42.218076,18.865091
2020-04-30,41.824989,123.000000,5000,54.172652,22.930132


In [26]:
# 各支股票淨值計算
preNAV_name_list=[]
for k in range(len(stock)):
    preNAV_list=[0]
    preNAV_name_list.append("preNAV"+str(k+1))
    for i,j in enumerate(stock_m_df[stock_m_df.columns[k]].iloc[1:]):
        preNAV=stock_m_df[unit_name_list[k]].iloc[i]*j
        preNAV_list.append(preNAV)
    stock_m_df[preNAV_name_list[k]]=preNAV_list

stock_m_df.head()

,KO,PYPL,total_cost,unit1,unit2,preNAV1,preNAV2
Date,,,,,,,
2019-12-31,50.008430,108.169998,1000,9.998314,4.622354,0.000000,0.000000
2020-01-31,52.764091,113.889999,2000,19.474456,9.012555,527.551965,526.439869
2020-02-28,48.327934,107.989998,3000,29.820439,13.642613,941.160217,973.265782
2020-03-31,40.330265,95.739998,4000,42.218076,18.865091,1202.666210,1306.143769
2020-04-30,41.824989,123.000000,5000,54.172652,22.930132,1765.770593,2320.406189


In [27]:

stock_m_df["final"]=stock_m_df[preNAV_name_list[0]].copy()
for i in range(1,len(stock)):
    stock_m_df["final"]+=stock_m_df[preNAV_name_list[i]]
for i in range(len(stock_m_df)):
    stock_m_df["final"][i]+=PMT
stock_m_df

C:\Users\PHIL\AppData\Local\Temp\ipykernel_5580\2403098464.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_m_df["final"][i]+=PMT


,KO,PYPL,total_cost,unit1,unit2,preNAV1,preNAV2,final
Date,,,,,,,,
2019-12-31,50.008430,108.169998,1000,9.998314,4.622354,0.000000,0.000000,1000.000000
2020-01-31,52.764091,113.889999,2000,19.474456,9.012555,527.551965,526.439869,2053.991834
2020-02-28,48.327934,107.989998,3000,29.820439,13.642613,941.160217,973.265782,2914.425999
2020-03-31,40.330265,95.739998,4000,42.218076,18.865091,1202.666210,1306.143769,3508.809978
2020-04-30,41.824989,123.000000,5000,54.172652,22.930132,1765.770593,2320.406189,5086.176782
2020-05-29,42.545006,155.009995,6000,65.924913,26.155730,2304.775795,3554.399576,6859.175371
2020-06-30,41.092129,174.229996,7000,78.092693,29.025500,2708.995003,4557.112740,8266.107743
2020-07-31,43.446560,196.070007,8000,89.601084,31.575610,3392.858873,5691.029996,10083.888869
2020-08-31,45.552670,204.139999,9000,100.577389,34.024909,4081.568578,6445.844915,11527.413493


In [28]:
# monthly_return
stdev=stock_m_df.copy()
monthly_return=[(stdev["final"][0]/PMT)-1]
for i in range(1,len(stdev)):
    mreturn=((stdev["final"][i]-PMT)/stdev["final"][i-1])-1
    monthly_return.append(mreturn*100)

stock_m_df["monthly return(%)"]=monthly_return
# stock_m_df["monthly return(%)"]=round(dfm["monthly return(%)"],2)

# CAGR
CAGR=round(((stock_m_df['final'][-1]/stock_m_df['total_cost'][0])**(1/((len(stock_m_df)-1)/12))-1)*100,2)
stock_m_df

,KO,PYPL,total_cost,unit1,unit2,preNAV1,preNAV2,final,monthly return(%)
Date,,,,,,,,,
2019-12-31,50.008430,108.169998,1000,9.998314,4.622354,0.000000,0.000000,1000.000000,0.000000
2020-01-31,52.764091,113.889999,2000,19.474456,9.012555,527.551965,526.439869,2053.991834,5.399183
2020-02-28,48.327934,107.989998,3000,29.820439,13.642613,941.160217,973.265782,2914.425999,-6.794858
2020-03-31,40.330265,95.739998,4000,42.218076,18.865091,1202.666210,1306.143769,3508.809978,-13.917527
2020-04-30,41.824989,123.000000,5000,54.172652,22.930132,1765.770593,2320.406189,5086.176782,16.454775
2020-05-29,42.545006,155.009995,6000,65.924913,26.155730,2304.775795,3554.399576,6859.175371,15.198028
2020-06-30,41.092129,174.229996,7000,78.092693,29.025500,2708.995003,4557.112740,8266.107743,5.932672
2020-07-31,43.446560,196.070007,8000,89.601084,31.575610,3392.858873,5691.029996,10083.888869,9.893183
2020-08-31,45.552670,204.139999,9000,100.577389,34.024909,4081.568578,6445.844915,11527.413493,4.398349


In [29]:
def MDD(dr):
#     dr=df.pct_change(1)
    r=dr.add(1).cumprod()
    dd=r.div(r.cummax()).sub(1)
    mdd=dd.min()
    end=dd.idxmin()
    start=r.loc[:end[0]].idxmax()
    days=end-start
    return mdd[0], start[0], end[0], days[0]

# MAX Drawdown


In [30]:
mdd=(stock_m_df["monthly return(%)"][0:]/100).copy()
mdd=mdd.to_frame()
MDD(mdd)
# print(ddddd)

(-0.39621486630318514,
 Timestamp('2021-08-31 00:00:00'),
 Timestamp('2022-09-30 00:00:00'),
 Timedelta('395 days 00:00:00'))

# stdev

In [31]:
df_stdev=stock_m_df.copy()
# df_stdev=df_stdev["monthly return(%)"]
risk=df_stdev["monthly return(%)"][1:].std()
risk_Y=risk*12**0.5
risk_Y=round(risk_Y,2)
ar=df_stdev["monthly return(%)"][1:].mean()*12
std=risk_Y
# Sharpe
sp=ar/std
sp=round(sp,2)
# sortino
sortino=[]
for i in range(1,len(df_stdev)):
    if df_stdev["monthly return(%)"][i]<0:
        sortino.append(df_stdev["monthly return(%)"][i])
    else:
        sortino.append(0)

std_D=DataFrame(sortino).std()*12**0.5
st=ar/std_D
st=round(st,3)



print("年化標準差:",risk_Y,"%")
print('Sharpe ratio:',sp)
print('sortino ratio:',st[0])


# df_stdev.plot()

年化標準差: 26.57 %
Sharpe ratio: 0.28
sortino ratio: 0.503


In [32]:
stock_m_df_c=stock_m_df.copy()
stock_m_df_c['final']=round(stock_m_df_c['final'])

# stock_m_df_c

In [34]:
print("========================================================")
print("標的:",stock)
print("回測區間:",start,'~',end)
print("總損益:",stock_m_df_c['final'][-1])
print("CAGR:",CAGR,'%')
print("標準差:","{:.2f}".format(risk_Y),"%")
print('Sharpe ratio:',sp)
print('sortino ratio:',st[0])
print('最大回撤(MAX Drawdown):',round(MDD(mdd)[0]*100,2),'%')
print('最大回撤區間:',MDD(mdd)[1].strftime('%Y-%m-%d'),'~',MDD(mdd)[2].strftime('%Y-%m-%d'))
print("========================================================")


標的: ['KO', 'PYPL']
回測區間: 2020-1-1 ~ 2022-12-31
總損益: 32319.0
CAGR: 218.53 %
標準差: 26.57 %
Sharpe ratio: 0.28
sortino ratio: 0.503
最大回撤(MAX Drawdown): -39.62 %
最大回撤區間: 2021-08-31 ~ 2022-09-30


In [35]:
stock_m_df_2=stock_m_df.copy()
final_l=stock_m_df_2['final'].copy()
for i in range(len(stock_m_df_2)):
    final_l[i]=PMT


for i in range(1,len(stock_m_df_2)):
    for j in range(len(stock)):
        stock_m_df_2[preNAV_name_list[j]].iloc[i]=stock_m_df_2[unit_name_list[j]].iloc[i-1]*stock_m_df_2[stock[j]].iloc[i]
        final_l[i]+=stock_m_df_2[preNAV_name_list[j]].iloc[i]
    stock_m_df_2['final'][i]=final_l[i]    
    for j in range(len(stock)):        
        stock_m_df_2[unit_name_list[j]].iloc[i]=(((stock_m_df_2[preNAV_name_list[j]].iloc[i]/(stock_m_df_2['final'].iloc[i]-PMT))*PMT)/stock_m_df_2[stock[j]].iloc[i])
        stock_m_df_2[unit_name_list[j]].iloc[i]+=stock_m_df_2[unit_name_list[j]].iloc[i-1]
     
        
stock_m_df_2

C:\Users\PHIL\AppData\Local\Temp\ipykernel_5580\3029264020.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_m_df_2[preNAV_name_list[j]].iloc[i]=stock_m_df_2[unit_name_list[j]].iloc[i-1]*stock_m_df_2[stock[j]].iloc[i]
C:\Users\PHIL\AppData\Local\Temp\ipykernel_5580\3029264020.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_m_df_2['final'][i]=final_l[i]
C:\Users\PHIL\AppData\Local\Temp\ipykernel_5580\3029264020.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

,KO,PYPL,total_cost,unit1,unit2,preNAV1,preNAV2,final,monthly return(%)
Date,,,,,,,,,
2019-12-31,50.008430,108.169998,1000,9.998314,4.622354,0.000000,0.000000,1000.000000,0.000000
2020-01-31,52.764091,113.889999,2000,19.484454,9.007923,527.551965,526.439869,2053.991834,5.399183
2020-02-28,48.327934,107.989998,3000,29.662245,13.713251,941.643425,972.765548,2914.408974,-6.794858
2020-03-31,40.330265,95.739998,4000,41.483674,19.178455,1196.286211,1312.906614,3509.192825,-13.917527
2020-04-30,41.824989,123.000000,5000,51.616462,23.862978,1735.054237,2358.949968,5094.004205,16.454775
2020-05-29,42.545006,155.009995,6000,60.372401,27.910965,2196.022666,3699.000023,6895.022689,15.198028
2020-06-30,41.092129,174.229996,7000,68.593315,31.711603,2480.830485,4862.927309,8343.757794,5.932672
2020-07-31,43.446560,196.070007,8000,76.050863,35.159327,2980.143580,6217.694229,10197.837809,9.893183
2020-08-31,45.552670,204.139999,9000,83.197329,38.463233,3464.319832,7177.424918,11641.744750,4.398349


In [36]:
# monthly_return
stdev_2=stock_m_df_2.copy()
monthly_return=[(stdev_2["final"][0]/PMT)-1]
for i in range(1,len(stdev_2)):
    mreturn=((stdev_2["final"][i]-PMT)/stdev_2["final"][i-1])-1
    monthly_return.append(mreturn*100)

stock_m_df_2["monthly return(%)"]=monthly_return

# stock_m_df_2

In [37]:
df_stdev_2=stock_m_df_2.copy()
# df_stdev=df_stdev["monthly return(%)"]
risk_2=df_stdev_2["monthly return(%)"][1:].std()
risk_Y_2=risk_2*12**0.5
risk_Y_2=risk_Y_2
ar_2=df_stdev_2["monthly return(%)"][1:].mean()*12
std_2=risk_Y_2
sp_2=ar_2/std_2
sp_2=round(sp_2,2)
# sortino
sortino_2=[]
for i in range(1,len(df_stdev_2)):
    if df_stdev_2["monthly return(%)"][i]<0:
        sortino_2.append(df_stdev_2["monthly return(%)"][i])
    else:
        sortino_2.append(0)
# print(sortino,len(sortino),DataFrame(sortino).std())
sttd_D_2=DataFrame(sortino_2).std()*12**0.5
stt_2=ar_2/sttd_D_2
stt_2=round(stt_2,3)
df_stdev_2["monthly return(%)"][1:]
std_D_2=df_stdev_2["monthly return(%)"][1:][df_stdev_2["monthly return(%)"]<0].std()*12**0.5
st_2=ar_2/std_D_2
st_2=round(st_2,3)


print("年化標準差:",risk_Y_2,"%")
print('Sharpe ratio:',sp_2)
print('sortino ratio:',stt_2[0])
print('sortino ratio:',st_2)


年化標準差: 29.391681438350645 %
Sharpe ratio: 0.1
sortino ratio: 0.17
sortino ratio: 0.166


In [38]:
stock_m_df_2_c=stock_m_df_2.copy()

for i in range(len(stock)):
    stock_m_df_2_c[stock[i]]=round(stock_m_df_2_c[stock[i]],2)
    stock_m_df_2_c[preNAV_name_list[i]]=round(stock_m_df_2_c[preNAV_name_list[i]])
    stock_m_df_2_c['final']=round(stock_m_df_2_c['final'])
    stock_m_df_2_c['monthly return(%)']=round(stock_m_df_2_c['monthly return(%)'],2)
stock_m_df_2_c

,KO,PYPL,total_cost,unit1,unit2,preNAV1,preNAV2,final,monthly return(%)
Date,,,,,,,,,
2019-12-31,50.01,108.17,1000,9.998314,4.622354,0.0,0.0,1000.0,0.00
2020-01-31,52.76,113.89,2000,19.484454,9.007923,528.0,526.0,2054.0,5.40
2020-02-28,48.33,107.99,3000,29.662245,13.713251,942.0,973.0,2914.0,-6.80
2020-03-31,40.33,95.74,4000,41.483674,19.178455,1196.0,1313.0,3509.0,-13.90
2020-04-30,41.82,123.00,5000,51.616462,23.862978,1735.0,2359.0,5094.0,16.67
2020-05-29,42.55,155.01,6000,60.372401,27.910965,2196.0,3699.0,6895.0,15.72
2020-06-30,41.09,174.23,7000,68.593315,31.711603,2481.0,4863.0,8344.0,6.51
2020-07-31,43.45,196.07,8000,76.050863,35.159327,2980.0,6218.0,10198.0,10.24
2020-08-31,45.55,204.14,9000,83.197329,38.463233,3464.0,7177.0,11642.0,4.35


In [39]:
mdd_2=(stock_m_df_2["monthly return(%)"][0:]/100).copy()
mdd_2=mdd_2.to_frame()
MDD(mdd_2)

(-0.49797869319003474,
 Timestamp('2021-08-31 00:00:00'),
 Timestamp('2022-06-30 00:00:00'),
 Timedelta('303 days 00:00:00'))

In [40]:

CAGR_2=round(((stock_m_df_2['final'][-1]/stock_m_df_2['total_cost'][0])**(1/((len(stock_m_df_2)-1)/12))-1)*100,2)
print("========================================================")
print("標的:",stock)
print("回測區間:",start,'~',end)
print("總損益:",round(stock_m_df_2['final'][-1]))
print("CAGR:",CAGR_2,'%')
print("標準差:","{:.2f}".format(risk_Y_2),"%")
print('Sharpe ratio:',sp_2)
print('Sortino ratio:',stt_2[0])
print('最大回撤(MAX. Drawdown):',round(MDD(mdd_2)[0]*100,2),'%')
print('最大回撤區間:',MDD(mdd_2)[1].strftime('%Y-%m-%d'),'~',MDD(mdd_2)[2].strftime('%Y-%m-%d'))
print("========================================================")


標的: ['KO', 'PYPL']
回測區間: 2020-1-1 ~ 2022-12-31
總損益: 28813
CAGR: 206.57 %
標準差: 29.39 %
Sharpe ratio: 0.1
Sortino ratio: 0.17
最大回撤(MAX. Drawdown): -49.8 %
最大回撤區間: 2021-08-31 ~ 2022-06-30


In [41]:
stock_m_df_2_c

,KO,PYPL,total_cost,unit1,unit2,preNAV1,preNAV2,final,monthly return(%)
Date,,,,,,,,,
2019-12-31,50.01,108.17,1000,9.998314,4.622354,0.0,0.0,1000.0,0.00
2020-01-31,52.76,113.89,2000,19.484454,9.007923,528.0,526.0,2054.0,5.40
2020-02-28,48.33,107.99,3000,29.662245,13.713251,942.0,973.0,2914.0,-6.80
2020-03-31,40.33,95.74,4000,41.483674,19.178455,1196.0,1313.0,3509.0,-13.90
2020-04-30,41.82,123.00,5000,51.616462,23.862978,1735.0,2359.0,5094.0,16.67
2020-05-29,42.55,155.01,6000,60.372401,27.910965,2196.0,3699.0,6895.0,15.72
2020-06-30,41.09,174.23,7000,68.593315,31.711603,2481.0,4863.0,8344.0,6.51
2020-07-31,43.45,196.07,8000,76.050863,35.159327,2980.0,6218.0,10198.0,10.24
2020-08-31,45.55,204.14,9000,83.197329,38.463233,3464.0,7177.0,11642.0,4.35


In [ ]:
# return_name_list=[]
# cc=[]
# for k in range(len(dfm.columns[0:len(stock_list)])):
#     return_name_list.append("return"+str(k+1)+'(%)')
# #     dfm[cc[k]]=(dfm[aa[k]]/(dfm['total_cost']*ratio[k]-PMT*ratio[k])-1)*100
#     dfm[return_name_list[k]]=round((dfm[preNAV_name_list[k]]/(dfm['total_cost']*ratio[k]-PMT*ratio[k])-1)*100,2)

# dfm.head()

In [ ]:
# T_R=TR/100
# print('總回報',TR,'%')
# months=len(dfm)
# print("共有:",months,'月')
# IRR=(1+T_R)**(12/months)-1
# print('內部報酬率(IRR):',round(IRR*100,2),'%')